In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import time 
import os
import h5py

In [10]:
def print_results(video, limit=None):
        #fig=plt.figure(figsize=(16, 30))
        if not os.path.exists('output'):
            os.mkdir('output')

        print("Loading model ...")
        model = load_model('modelnew.h5')
        Q = deque(maxlen=128)
        vs = cv2.VideoCapture(video)
        writer = None
        (W, H) = (None, None)
        count = 0     
        while True:
            # read the next frame from the file
            (grabbed, frame) = vs.read()

            print(grabbed)

            # if the frame was not grabbed, then we have reached the end
            # of the stream
            if not grabbed:
                break
            
            # if the frame dimensions are empty, grab them
            if W is None or H is None:
                (H, W) = frame.shape[:2]

            # clone the output frame, then convert it from BGR to RGB
            # ordering, resize the frame to a fixed 128x128, and then
            # perform mean subtraction

            print(H, W)
            output = frame.copy()
           
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (128, 128)).astype("float32")
            frame = frame.reshape(128, 128, 3) / 255

            # make predictions on the frame and then update the predictions
            # queue
            preds = model.predict(np.expand_dims(frame, axis=0))[0]
            print("preds",preds)
            Q.append(preds)

            results = np.array(Q).mean(axis=0)
            i = (preds > 0.50)[0]
            label = i

            text_color = (0, 255, 0)

            if label:
                text_color = (0, 0, 255) 

            else:
                text_color = (0, 255, 0)

            text = "Violence: {}".format(label)
            FONT = cv2.FONT_HERSHEY_SIMPLEX 

            cv2.putText(output, text, (35, 50), FONT,1.25, text_color, 3) 

            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter("output/v_output.avi", fourcc, 30,(W, H), True)

            # write the output frame to disk
            writer.write(output)

            cv2.imshow("OUTPUT", output)
            key = cv2.waitKey(1) & 0xFF

            if key == ord("q"):
                break
        
        print("[INFO] cleaning up...")
        # writer.release()
        vs.release()
        cv2.destroyAllWindows()

In [14]:
V_path = "D:/KAIF/RJPOLICE_HACK_587_NeuralEnvoys_3/Models/ViolenceDetection/vids/fight03.mp4"  
NV_path = "D:/Kaif/Hackathon/Models/ViolenceDetection/vids/soccer.mp4"

In [15]:
print_results(V_path)

Loading model ...
True
360 640
1/1 [==============================] - 0s 442ms/step
preds [0.00214133]
True
360 640
1/1 [==============================] - 0s 20ms/step
preds [0.00161418]
True
360 640
1/1 [==============================] - 0s 23ms/step
preds [0.00116515]
True
360 640
1/1 [==============================] - 0s 17ms/step
preds [0.00087452]
True
360 640
1/1 [==============================] - 0s 9ms/step
preds [0.00067813]
True
360 640
1/1 [==============================] - 0s 20ms/step
preds [0.00088755]
True
360 640
1/1 [==============================] - 0s 16ms/step
preds [0.00169487]
True
360 640
1/1 [==============================] - 0s 15ms/step
preds [0.00565119]
True
360 640
1/1 [==============================] - 0s 19ms/step
preds [0.00347081]
True
360 640
1/1 [==============================] - 0s 26ms/step
preds [0.00297183]
True
360 640
1/1 [==============================] - 0s 17ms/step
preds [0.00469341]
True
360 640
1/1 [==============================] - 0s 6ms

In [ ]:
print_results(NV_path)

## Logs on console

In [9]:
import datetime

def print_results(video, limit=None, file_name=None):
    if not os.path.exists('output'):
        os.mkdir('output')

    model = load_model('modelnew.h5')
    Q = deque(maxlen=128)
    vs = cv2.VideoCapture(video)
    writer = None
    (W, H) = (None, None)
    count = 0
    consecutive_frames = 0
    
    while True:
        (grabbed, frame) = vs.read()

        if not grabbed:
            break
        
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255

        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)
        results = np.array(Q).mean(axis=0)
        i = (preds > 0.30)[0]

        if i:
            consecutive_frames += 1
            if consecutive_frames == 3:
                current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                print(f"Violence detected | {file_name if file_name else 'Unknown'} | {current_time} | Probability: {preds}")
                consecutive_frames = 0  # Reset the count after logging
        else:
            consecutive_frames = 0

        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter("output/v_output.avi", fourcc, 30, (W, H), True)

        writer.write(output)

        cv2.imshow("OUTPUT", output)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
        
    print("[INFO] cleaning up...")
    # writer.release()
    vs.release()
    cv2.destroyAllWindows()

# Example usage:
V_path = "D:/Kaif/Hackathon/Models/ViolenceDetection/vids/fight01.mp4"
NV_path = "D:/Kaif/Hackathon/Models/ViolenceDetection/vids/soccer.mp4"

print_results(V_path, file_name="yt1.mp4")

[INFO] cleaning up...


## Log to CSV file

In [5]:
import cv2
import numpy as np
import os
from collections import deque
from datetime import datetime
import csv

def print_results(video, limit=None):
    if not os.path.exists('output'):
        os.mkdir('output')

    if not os.path.exists('logs'):
        os.mkdir('logs')

    print("Loading model ...")
    model = load_model('modelnew.h5')
    Q = deque(maxlen=128)
    vs = cv2.VideoCapture(video)
    writer = None
    (W, H) = (None, None)
    count = 0
    consecutive_detection = 0 
    current_date = datetime.now().strftime("%Y-%m-%d")
    logs_folder = 'logs'
    log_filename = f"{current_date}_logfile.csv"
    log_path = os.path.join(logs_folder, log_filename)

    with open(log_path, mode='a', newline='') as file:
        csv_writer = csv.writer(file)

        if os.path.getsize(log_path) == 0:
            csv_writer.writerow(['Violence Detected', 'Filename/Cam Name', 'Time and Date', 'Probability'])

        while True:
            (grabbed, frame) = vs.read()

            if not grabbed:
                break

            if W is None or H is None:
                (H, W) = frame.shape[:2]

            output = frame.copy()

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (128, 128)).astype("float32")
            frame = frame.reshape(128, 128, 3) / 255

            preds = model.predict(np.expand_dims(frame, axis=0))[0]
            Q.append(preds)

            results = np.array(Q).mean(axis=0)
            i = (preds > 0.50)[0]

            if i:
                consecutive_detection += 1
                if consecutive_detection >= 3:
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    csv_writer.writerow(['Yes', log_filename if log_filename else 'Unknown', timestamp, preds[0]])
                    consecutive_detection = 0  

            else:
                consecutive_detection = 0  

            text = "Violence: {}".format(i)
            FONT = cv2.FONT_HERSHEY_SIMPLEX 

            text_color = (0, 255, 0) 
            if i:
                text_color = (0, 0, 255)

            cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter(os.path.join('output', f"{current_date}_v_output.avi"), fourcc, 30, (W, H), True)

            writer.write(output)

            cv2.imshow("OUTPUT", output)
            key = cv2.waitKey(1) & 0xFF

            if key == ord("q"):
                break

    print("[INFO] cleaning up...")
    if writer:
        writer.release()
    vs.release()
    cv2.destroyAllWindows()

In [ ]:
print_results("D:/Kaif/Hackathon/Models/ViolenceDetection/vids/fight04.mp4")